<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

Se va a implementar un algoritmo para resolver la suma de matrices de dimension N*N. 

Se implementaran dos algoritmos, uno ejecuta y resuelve el calculo de manera secuencial sobre el mismo procesador y el segundo implementa la suma de los terminos de la matriz de forma paralela utilizando GPU. Para esto se implementan hilos en dos dimensiones, a partir de un calulo en el kernel podemos acceder a cada elemento de la matriz para poder realizar la operacion.


#1.1 Explicación del algoritmo

La suma de matrices[1] es una operación lineal que consiste en unificar los elementos de dos o más matrices que coincidan en posición dentro de sus respectivas matrices y que estas tengan el mismo orden.

En otras palabras, el sumatorio de una o más matrices es la unión de los elementos que tengan la misma posición dentro de las matrices y que estas tengan el mismo orden.

![](https://drive.google.com/uc?export=view&id=16M9RJRRsQELQqiC15qRqNC_bdQBfMc1_)

Para sumar dos matrices se debe verificar que el orden de estas sean iguales.

#2. Armado del ambiente

##2.1 Armado del ambiente en CPU

No se requiere la ejecucion previa de algun comando

##2.1 Armado del ambiente en GPU

1. Ir a Entornto de ejecucion -> Cambiar tipo de entorno de ejecucion y seleccionar la opcion GPU.
2. Se requiere la instalación del modulo de Cuda para Python.

In [1]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 9.2MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621146 sha256=873b09dd6b3609a0738433e6fdf6549bf2f6f3b834feb39642d526f014af3e5c
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=69ac9e60bd1c1674e6bcd5d8bbe4868231d5c87d559ddb09844f20af1cb3dd63
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

##3.1 Desarrollo en CPU

In [6]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  -1#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import sys

tiempoTotal = time.time() * 1000
try: 
  matriz_A = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A = matriz_A.astype(numpy.int32())
  matriz_B = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B = matriz_B.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño no puede ser negativo")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B)
print("---------------------------------------------------")

tiempoBucle = time.time() * 1000
matriz_R = numpy.zeros((tamMatriz, tamMatriz))  
matriz_R = matriz_R.astype(numpy.int32())

for i in range (0, tamMatriz):
  for j in range(0, tamMatriz):
    matriz_R[i][j] = matriz_A[i][j] + matriz_B[i][j]

print("Matriz R:\n",matriz_R)
print("---------------------------------------------------")

tiempoTotal = (time.time() * 1000) - tiempoTotal
tiempoBucle = (time.time() * 1000) - tiempoBucle
print("--------------------------------------------------")
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


##3.2 Desarrollo en GPU

In [16]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }
tamMatriz =  -0#@param {type: "number"}
# --------------------------------------------

import time

import numpy
import sys
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempoTotal = (time.time()*1000)

try: 
  matriz_A_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_A_cpu = matriz_A_cpu.astype(numpy.int32())
  matriz_B_cpu = numpy.random.randint(5000, size=(tamMatriz, tamMatriz))
  matriz_B_cpu = matriz_B_cpu.astype(numpy.int32())
except Exception:
  sys.exit("El tamaño de la matriz no puede ser negativo")

try:  
  matriz_R_cpu = numpy.zeros((tamMatriz, tamMatriz)) 
  matriz_R_cpu = matriz_R_cpu.astype(numpy.int32()) 
  matriz_A_gpu = cuda.mem_alloc(matriz_A_cpu.nbytes)
  matriz_B_gpu = cuda.mem_alloc(matriz_B_cpu.nbytes)
  matriz_R_gpu = cuda.mem_alloc(matriz_R_cpu.nbytes)
  cuda.memcpy_htod(matriz_A_gpu, matriz_A_cpu)
  cuda.memcpy_htod(matriz_B_gpu, matriz_B_cpu)
  cuda.memcpy_htod(matriz_R_gpu, matriz_R_cpu)
except Exception:
  sys.exit("Error de asignacion de memoria para gpu")

print("---------------------------------------------------")
print("Matriz A:\n",matriz_A_cpu)
print("---------------------------------------------------")
print("Matriz B:\n",matriz_B_cpu)
print("---------------------------------------------------")

module = SourceModule("""
__global__ void matrixMultiplicationKernel(float* A, float* B, float* R,
                                           int N
                                           ) {

    int x = blockIdx.y*blockDim.y+threadIdx.y;
    int y = blockIdx.x*blockDim.x+threadIdx.x;

    if(y < N){
      R[x+ y*N] = A[x+ y*N]  +  B[x+ y*N];
    }
}
""")
kernel = module.get_function("matrixMultiplicationKernel")

dim_hilo_x = 16
dim_bloque_x = numpy.int( (tamMatriz + dim_hilo_x-1) / dim_hilo_x )

dim_hilo_y = 16
dim_bloque_y = numpy.int( (tamMatriz + dim_hilo_y-1) / dim_hilo_y )

print( "Thread: [", dim_hilo_x, ",", dim_hilo_y, " ], Bloque : [", dim_bloque_x, ",", dim_bloque_y, "]" )
print( "Total de Thread: [", dim_hilo_x*dim_bloque_x, ",", dim_hilo_y*dim_bloque_y, " ]", " = ", dim_hilo_x*dim_bloque_x*dim_hilo_y*dim_bloque_y )

tiempo_kernel = time.time() * 1000

kernel(matriz_A_gpu, matriz_B_gpu, matriz_R_gpu,
       numpy.int32(tamMatriz),        
       block=( dim_hilo_x, dim_hilo_y, 1), 
       grid=(dim_bloque_x, dim_bloque_y,1)
       )

tiempo_kernel = (time.time() *1000) - tiempo_kernel

cuda.memcpy_dtoh(matriz_R_cpu, matriz_R_gpu)
tiempoTotal = (time.time() * 1000) - tiempoTotal
print("---------------------------------------------------")
print("Matriz resultado:\n",matriz_R_cpu)
print("---------------------------------------------------")
print("Tiempo TOTAL: ", tiempoTotal, "[ms]" )
print("Tiempo GPU  : ", tiempo_kernel, "[ms]" )

SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#4. Tabla de pasos

##4.1 Tabla de pasos CPU

Tabla de ejecucion de forma secuencial
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|tiempoBucle	       |Se toma el tiempo inicial del algoritmo presentado.
>CPU	|for…	for            |Se realiza el algoritmo.
>CPU	|tiempoBucle	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

##4.1 Tabla de pasos GPU

Tabla de ejecucion en multiples hilos de ejecucion
>Procesador | Funcion | Detalle
>--- | --- | ----
>CPU	|@param              |Lectura del tamaño de matriz.
>CPU	|import              |Se importa los diferentes modulos a utilizar.
>CPU	|tiempoTotal         |Se obtiene el tiempo actual para futuras mediciones.
>CPU	|numpy.random.randint|Inicializacion de matrices.
>CPU	|numpy.zeros         |Inicializacion de matriz de resultado.
>**GPU**  |cuda.mem_alloc        |Reserva de memoria de GPU.
>**GPU**  |cuda.memcpy_htod      |Copia de memoria de CPU a GPU.
>CPU	|SourceModule        |Definición de código del kernel.
>CPU	|module.get_function        |Generación de función del Kernel de GPU.
>CPU	|dim_hilo_x, dim_hilo_y	    | Calcula las dimensiones para la ejecuciòn de 2D.
>CPU	|tiempo_kernel	       |Toma el tiempo inicial del algoritmo presentado.
>**GPU**	|kernel()            |Ejecución del kernel
>CPU	|tiempo_kernel	       |Toma el tiempo final del algoritmo presentado.
>CPU	|tiempoTotal	       |Toma el tiempo final de la ejecucion.
>CPU	|print()	           |Se informan los datos.

#5. Conclusiones

El algoritmo no presenta mucha dificultad, lo interesante es que el primer algoritmo posee una complejidad O(N^2) y su rendimiento empeorara a medida que las dimensiones de la matriz sean mayores. Con la implementación paralela aumenta la eficiencia frente a la ejecución secuencial.

#6. Bibliografía

[1].Suma de matrices [matrices](https://economipedia.com/definiciones/suma-de-matrices.html)